In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import utils
from feature_engineering import codify_date, codify_date_2, remove_outliers, get_X_y, covid_19, covid_19_2
from feature_engineering import add_weather, add_lag_and_rolling_features
from utils import handle_missing_values
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn import linear_model
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
from sklearn.preprocessing import FunctionTransformer


def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

In [ ]:
data = pd.read_parquet(Path("data") / "train.parquet")
mdata = codify_date_2(data)
mdata = remove_outliers(mdata)
mdata = covid_19(mdata)
mdata = add_weather(mdata)
mdata = handle_missing_values(mdata, "linear")
X, y = get_X_y(mdata)
X.head()

In [ ]:
X_test = pd.read_parquet(Path("data") / "final_test.parquet")
X_test = codify_date_2(X_test)
X_test = covid_19(X_test)
X_test = add_weather(X_test)
X_test = handle_missing_values(X_test, "linear")

X_test.head()

In [ ]:
# Save original index
X_original = pd.read_parquet(Path("data") / "final_test.parquet")
X_original.loc[:, "index1"] = X_original.index
X_original = codify_date_2(X_original)

# Merge DataFrames
X_test.loc[:, "index2"] = X_test.index
merged_df = X_test.merge(X_original, on=["datetime", "counter_name"], how="left")

merged_df = merged_df.sort_values("index1")
display(merged_df[["index1", "index2"]])


In [ ]:
X_test = X_test.drop(columns=["counter_id", "site_id", "site_name", 
                                "counter_installation_date", 
                              "coordinates", "counter_technical_id",
                              "latitude", "longitude", "datetime", "date"])

In [ ]:
numerical_columns = ['t', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend']
categorical_columns = ["counter_name", "year", "month", "day", "day_of_week", "is_weekend", "IsHoliday"]

preprocessor = ColumnTransformer(transformers=[
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns),
    ("hour_sin", sin_transformer(24), ["hour"]),
    ("hour_cos", cos_transformer(24), ["hour"]),
], remainder='passthrough')

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor())
])

# Fit the pipeline to the training data
pipeline.fit(X, y)

y_pred = pipeline.predict(X_test)
y_pred = y_pred[merged_df["index2"]]

results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission_maxim.csv", index=False)